# ADM Homework 2 - Research Questions (RQs)

## Preliminary actions

This section of the notebook contains all the cleaning operations on the datasets. Originally, they have been implemented while solving the research questions. Every question made us work on different attributes of the datasets and allowed us to understand in detail all the characteristics of the attributes, such as anomalies and empty values. For clarity issues, we decided to perform most of these operations at the beginning of the notebook under the "Preliminary actions" section.

In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import math 
import calendar
import re
import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

In [3]:
df_1 = pd.read_json('lighter_authors.json', lines = True)

In [9]:
chunk = pd.read_json("lighter_books.json", lines = True, chunksize = 100000)
df_2 = []
for i in chunk:
    i = i.drop(["isbn"], axis = 1)
    i = i.drop(["authors"], axis = 1)
    i = i.drop(["isbn13"], axis = 1)
    i = i.drop(["image_url"], axis = 1)
    i = i.drop(["description"], axis = 1)
    i = i.drop(["shelves"], axis = 1)
    i = i.drop(["asin"], axis = 1)
    i = i.drop(["series_position"], axis = 1)
    df_2.append(i)
df_2 = pd.concat(df_2, ignore_index = True)

This code has been implemented in order to manage the original 'ligher_books' file, that was impossible to open in its original version, because of memory issues. It reads the file in chunks of 100.000 rows at a time while we remove 8 columns to make the whole dataset lighter. The average time to open it with this method is around 10 minutes.

**df_1 (lighter_authors) - preliminary cleaning**

In [4]:
#CLEANING 1: dropping not-needed columns
df_1.drop(['image_url','about'], axis = 1, inplace=True)

#CLEANING 2: operations on "name" column
df_1['name'] = df_1['name'].str.strip().replace(['Source Wikipedia', 'Source Wikia', 'Fuente Wikipedia'], 'Wikipedia')
df_1 = df_1[df_1.name != 'NOT A BOOK']
df_1 = df_1[df_1.name != 'Unknown']
df_1 = df_1[df_1.name != 'Various'] 
df_1 = df_1[df_1.name != 'Anonymous']

**df_2 (lighter_books) - preliminary cleaning**

In [10]:
#CLEANING 1: setting appropriate avararge rating interval
df_2 = df_2[ (df_2.average_rating >= 1) & (df_2.average_rating <= 5)]

#CLEANING 2: operations on "name" column
df_2 = df_2[df_2.author_name != 'NOT A BOOK']
df_2 = df_2[df_2.author_name != 'Unknown']
df_2 = df_2[df_2.author_name != 'Various']
df_2 = df_2[df_2.author_name != 'Anonymous']
df_2['author_name'] = df_2['author_name'].str.strip().replace(['Source Wikipedia', 'Source Wikia', 'Fuente Wikipedia'], 'Wikipedia')

#CLEANING 3: converting "num_pages" values into numbers
df_2['num_pages'] = pd.to_numeric(df_2['num_pages'], errors='coerce').fillna(0).astype(np.int64)
df_2 = df_2[df_2['num_pages'] > 0]

#CLEANING 4: operations on "rating_dist" column
df_2['rating_dist'] = df_2['rating_dist'].apply(lambda row: re.split(r':|\|', row))
df_2['rating_dist'] = df_2['rating_dist'].apply(lambda row: row[1::2])
df_2['rating_dist'] = df_2['rating_dist'].apply(lambda row: [int(e) for e in row])
keys=[5, 4, 3, 2, 1, 'total']
df_2['rating_dist'] = df_2['rating_dist'].apply(lambda row: dict(zip(keys, row)))

**CLEANING OPERATIONS - DF 1 (lighter_authors):**
1) We decided to drop the attributes "image_url" and "about" for memory issues and because we didn't need them;
2) The logic behind our choice on this part of the code is to consider "Wikipedia" as an author, while dropping "Unknown", "Various", "Anonymous" and "NOT A BOOK", not considering them as authors. 

**CLEANING OPERATIONS - DF 2 (lighter_books):**
1) The logic is the same for the point "1." of the previous cleaning operation;
2) The logic is the same for the point "3." of the previous cleaning operation;
3) We converted the values in "num_pages" into integers to be able to manipulate them;
4) We noticed that the "rating_dist" column was stored as a string, and we converted it into a dictionary to be able to access the informaion inside it in a convenient way.

# RQ 1 - *Exploratory Data Analysis (EDA)* 

## lighter_authors EDA

In [ ]:
#here we print all the attributes and the statistics of the dataset 
df_1.info()
summary_statistics_df_1 = df_1.describe()
print(summary_statistics_df_1)

In [ ]:
# here we print how a row looks like 
df_1.loc[0]

In [ ]:
#we search the max and min average ratings
max_avg_rating = df_1['average_rating'].max()
min_avg_rating = df_1['average_rating'].min()
print(max_avg_rating)
print(min_avg_rating)

#we find out that there is a rating that is -31 and this can't be possible because the ratings go from 1 to 5, so now we delete every rating that is <1

# setting appropriate avararge rating interval
df_1 = df_1[ (df_1.average_rating >= 1) & (df_1.average_rating <= 5)]

We noticed that there were values under the "average_rating" column outside the interval $[1, 5]$, which is the correct interval of accepted rating values, so we dropped everything outside it.

In [ ]:
plt.hist(df_1['average_rating'], bins=20, color='blue', edgecolor='black')
plt.xlabel('Average Ratings')
plt.ylabel('Frequency')
plt.title('Histogram of Average Ratings')
plt.show()

In [ ]:
plt.hist(df_1['text_reviews_count'], bins=20, color='blue', edgecolor='black')
plt.xlabel('Text reviews')
plt.ylabel('Frequency')
plt.title('Histogram of text reviews')
plt.yscale(value='log')
plt.show()

## lighter_books

In [ ]:
#here we print all the attributes and the statistics of the dataset 
df_2.info()
summary_statistics_df_2 = df_2.describe()
print(summary_statistics_df_2)

In [ ]:
#Let's try to find out if there are duplicate rows
columns_to_check = ['title']
duplicate_rows = df_2[df_2.duplicated(subset=columns_to_check, keep=False)]
num_duplicate_rows = duplicate_rows.shape[0]
print(f"Number of duplicate rows: {num_duplicate_rows}")
print(duplicate_rows.head(20))

# RQ2

## RQ2.1 Plot the number of books for each author in descending order

In [ ]:
df_rq21 = df_2[['title', 'author_id', 'work_id','author_name']]
df_rq21 = df_rq21.drop_duplicates()
author_books_count = df_rq21.groupby('author_id').agg({'work_id':'count', 'author_name':'first'}).reset_index()
author_books_count.rename(columns={'work_id': 'num_books', 'author_name':'author_name'}, inplace=True)
author_books_count = author_books_count.sort_values(by='num_books', ascending=False)
print("The number of books for each author in descending order is", author_books_count)

## RQ2.2 Which book has the highest number of reviews?

In [ ]:
max_text_reviews_count = df_2['text_reviews_count'].max()
max_row = df_2[df_2['text_reviews_count'] == max_text_reviews_count]
max_title = max_row['title'].values[0]
print("The book with the highest number of reviews is:", max_title, "with ", max_text_reviews_count, "reviews.")

## RQ2.3 Which are the top ten and ten worst books concerning the average score?

In [ ]:
# TOP 10

#all_5_ratings_books = len(df_2[df_2['average_rating']==5.0])
#we have seen that the books with an average rating of 5 are 231.276

df_2_ratings_5 = df_2[df_2['average_rating']==5.0]
df_highest_avg_score = df_2_ratings_5.nlargest(10, 'ratings_count')

df_highest_avg_score[['title', 'ratings_count']]

In [ ]:
# WORST 10

df_d = df_2[(df_2['average_rating'] <= 1) & (df_2['ratings_count'] > 0)]
df_lowest_avg_score = df_d.nlargest(10, 'ratings_count')

df_lowest_avg_score[['title', 'ratings_count', 'average_rating']]

## RQ2.4 Explore the different languages in the book’s dataset, providing a proper chart summarizing how these languages are distributed throughout our virtual library.

In [ ]:
df_2_languages = df_2[df_2['language']!='']
df_2_languages.loc[:,'language'] = df_2_languages['language'].replace(['en-US', 'en-GB'], 'eng')
language_counts = df_2_languages['language'].value_counts()
min_percentage = 1
languages = language_counts[language_counts / language_counts.sum() * 100 >= min_percentage]
table = pd.DataFrame({'Language': languages.index, 'Percentage': languages.values / language_counts.sum() * 100})
plt.figure(figsize=(12, 6))

plt.subplot(121)
plt.pie(languages, labels=languages.index, autopct='%1.1f%%')
plt.title("Languages Distribution")

plt.subplot(122)
plt.axis('off')
table_formatted = table.copy()
table_formatted['Percentage'] = table_formatted['Percentage'].apply(lambda x: f'{x:.1f}%')
table = plt.table(cellText=table_formatted.values, colLabels=table.columns, loc='center')

plt.show()

## RQ2.5 How many books have more than 250 pages?

In [ ]:
df_pages = df_2[df_2['num_pages'] >= 250]
print("The books with more than 250 pages are:", len(df_pages))

## RQ2.6 Plot the distribution of the fans count for the 50 most prolific authors (the ones who have written more books)

In [ ]:
#barra di wikipedia è più blu delle altre
df_prolific_authors = df_1.nlargest(50, 'works_count')
fans_count = df_prolific_authors['fans_count']
authors = df_prolific_authors['name']

plt.figure(figsize=(12, 6))
plt.bar(authors, fans_count, color='blue', alpha=0.7)
plt.yscale(value='log')
plt.xlabel('Authors')
plt.ylabel('Fans Count')
plt.title('Fans Distribution')
plt.xticks(rotation=90)
plt.show()

# RQ3

To answer this question, at the beginning we wanted to keep all the years of publication (starting from 0, cause there was a suggestion on slack to drop all the negative years of publication), but we noticed that for the first years there were a lot of misplaced informations (like book about people that weren't even born in that year, as one on William Shakespear "published" in year 0), so we decided to filter our dataset more. To do so, we decided to keep only the books published in or after 1455, which is the year of publication of Gutenberg's Bible, known to be the first ever book printed and published. For a more obvious reason, we also kept only the books published by the current year, 2023. 

In [ ]:
#creating a lighter df to solve this question 
df_rq3 = df_2[['title','num_pages','publication_date']]

#pre-processing 'publication_date' column 
pattern = r'(^[0-9]{4}-[0-9]{2}-[0-9]{1,2}$)|(^[0-9]{4}$)|(^[0-9]{4}-[0-9]{1,2}$)'
df_rq3= df_rq3[df_rq3['publication_date'].apply(lambda row: bool(re.search(pattern, row)))]
df_rq3['publication_date'] = df_rq3['publication_date'].apply(lambda row: re.split('-', row)) #this way we obtain year=df_rq3['original_publication_date'][0], month=df_rq3['original_publication_date'][1]
df_rq3['publication_year'] = df_rq3['publication_date'].apply(lambda row: int(row[0]))

#selecting books published in the range of years 1455-2023 as explained above
df_rq3= df_rq3[ df_rq3['publication_year'] <= 2023]
df_rq3= df_rq3[ df_rq3['publication_year'] >= 1455]

#cleaning data
df_rq3 = df_rq3[~df_rq3['publication_date'].isnull()]
df_rq3 = df_rq3[~df_rq3['publication_year'].isnull()]

## RQ3.1: Write a function that takes as input a year and returns as output the following information:   
- The number of books published that year.
- The total number of pages written that year.
- The most prolific month of that year.
- The longest book written that year.

In [ ]:
def rq3(year):

    if year in list(df_rq3['publication_year']):
        df_filtered = df_rq3[df_rq3['publication_year'] == year]
        
        #books published in that year
        books_published = len(df_filtered) 

        #total number of pages written 
        pages_written = df_filtered['num_pages'].sum()

        #most prolific mont
        #considering the books with the months registered
        hasmonth=[(len(date)>1) for date in df_filtered['publication_date']]
        
        if True in hasmonth:    
            df_filtered_withmonth = df_filtered[hasmonth]
            df_filtered_withmonth['month'] = df_filtered_withmonth.publication_date.apply(lambda row: int(row[1]))
            df_filtered_withmonth = df_filtered_withmonth[df_filtered_withmonth['month'] <= 12]
            
            grouped = df_filtered_withmonth.groupby(by='month')
            monthly_counts = grouped['title'].count()
            most_prolific_month = calendar.month_name[monthly_counts.idxmax()]
        #for the books with no month of publication registered
        else:  
            most_prolific_month= 'No publication month registered for this year'

        #longest book written in that year
        longest_book = df_filtered['title'][df_filtered['num_pages'].idxmax()]
        
        #output format:tuple
        return books_published, pages_written, most_prolific_month, longest_book

## RQ3.2: Use this function to build your data frame: the primary key will be a year, and the required information will be the attributes within the row. Finally, show the head and the tail of this new data frame considering the first ten years registered and the last ten years.

In [ ]:
first_year = df_rq3.publication_year.min()
last_year = 2023 

#getting a list with all the years registered in "publication_year" without duplicates
years = [ year for year in range(first_year, last_year + 1) if year in list(df_rq3['publication_year']) ] 


d = []
for year in years:
    attributes = rq3(year)
    d.append(
        {
            'books_published': attributes[0],
            'pages_written': attributes[1],
            'most_prolific_month':  attributes[2],
            'longest_book': attributes[3]
        }
    )

df_rq3_final = pd.DataFrame(d, index = years, columns= ['books_published', 'pages_written', 'most_prolific_month', 'longest_book'])

In [ ]:
df_rq3_final.head(10)

In [ ]:
df_rq3_final.tail(10)

## RQ3.3: Ask ChatGPT or any other LLM chatbot tool to implement this function and compare your work with the one the bot gave you as an answer. Does the chatbot implementation work? Please test it out and verify the correctness of the implementation, explaining the process you followed to prove it.

The one below is the code provided from ChatGPT 3.5 after the following request: 

"*I have a dataset created by reducing the files "books.json" and "authors.json" gotten from this website: https://www.kaggle.com/datasets/opalskies/large-books-metadata-dataset-50-mill-entries* \
*I have to fullfill this request: \
Write a function that takes as input a year and returns as output the following information:\
-The number of books published that year.\
-The total number of pages written that year.\
-The most prolific month of that year.\
-The longest book written that year.*

*Use this function to build your data frame: the primary key will be a year, and the required information will be the attributes within the row. Finally, show the head and the tail of this new data frame considering the first ten years registered and the last ten years.*

*How would you solve this question?*"

On a sidenote, we added the comment "## DO NOT RUN THIS CELL ##" at the beginning of the cell as a warning, of course it wasn't provided by ChatGPT.

In [ ]:
## DO NOT RUN THIS CELL ##

import pandas as pd
import json
# Load the "books.json" and "authors.json" data
books_data = pd.read_json("books.json")
authors_data = pd.read_json("authors.json")

# Merge the data to create a comprehensive dataset
data = pd.merge(books_data, authors_data, on="book_id")

# Define a function to get the required information for a specific year
def get_year_info(year):
    books_published = len(data[data["publication_date"].str.contains(str(year))])
    total_pages_written = data[data["publication_date"].str.contains(str(year))]["num_pages"].sum()
    most_prolific_month = data[data["publication_date"].str.contains(str(year))]["publication_date"].str.split('-').str[1].mode().iloc[0]
    longest_book = data[data["publication_date"].str.contains(str(year))]["num_pages"].idxmax()
    return books_published, total_pages_written, most_prolific_month, data.loc[longest_book]["title"]

# Create a DataFrame for the years, using the function to get the required information
years = range(2010, 2024)  # Assuming you want data for years 2010 to 2023
year_info = []

for year in years:
    info = get_year_info(year)
    year_info.append({"Year": year, "Books Published": info[0], "Total Pages Written": info[1], "Most Prolific Month": info[2], "Longest Book": info[3]})

year_df = pd.DataFrame(year_info)

# Display the head and tail of the new DataFrame
print("First ten years:")
print(year_df.head(10))
print("\nLast ten years:")
print(year_df.tail(10))    # Load the "books.json" and "authors.json" data
books_data = pd.read_json("books.json")
authors_data = pd.read_json("authors.json")

# Merge the data to create a comprehensive dataset
data = pd.merge(books_data, authors_data, on="book_id")

# Define a function to get the required information for a specific year
def get_year_info(year):
    books_published = len(data[data["publication_date"].str.contains(str(year))])
    total_pages_written = data[data["publication_date"].str.contains(str(year))]["num_pages"].sum()
    most_prolific_month = data[data["publication_date"].str.contains(str(year))]["publication_date"].str.split('-').str[1].mode().iloc[0]
    longest_book = data[data["publication_date"].str.contains(str(year))]["num_pages"].idxmax()
    return books_published, total_pages_written, most_prolific_month, data.loc[longest_book]["title"]

# Create a DataFrame for the years, using the function to get the required information
years = range(2010, 2024)  # Assuming you want data for years 2010 to 2023
year_info = []

for year in years:
    info = get_year_info(year)
    year_info.append({"Year": year, "Books Published": info[0], "Total Pages Written": info[1], "Most Prolific Month": info[2], "Longest Book": info[3]})

year_df = pd.DataFrame(year_info)

# Display the head and tail of the new DataFrame
print("First ten years:")
print(year_df.head(10))
print("\nLast ten years:")
print(year_df.tail(10))

First of all, let us say that the code doesn't work. There are many reasons why this appens. Here are the ones we spotted: 
- the files' names are wrong, but this error follows directly from the prompt I wrote: in that prompt I specifically mentioned the names of the files from which our dataset is obtained, but I ddidn't mention the current files' names;
- In line *9* of the code, there's a try to merge the authors' and the books' dataframe, but the specified column name (*book_id*) doesn't exist in any of the two datasets; morover this operation isn't necessary at all for the purpose of this question;
- In line *14*, the operation  *.sum()* won't work brecause the values of *'num_pages'* column are stored as strings, not as integers (or any numeric value), so they need to be converted to the right format before running this code;
- In line *15* there are at least two problems: when calling *.str[1]* we could get an error for the books that have no date of publication saved, or only the year of their publication and not the full date; when calling *.mode()* we won't get any value for the mode if there is only one publication for a given month, leading to wrong clasifications of the years in wich there's only one book per month with the full registered publication date;
- In line *27*, when creating the resulting final dataframe, there is no "index" parameter specified, so the resulting dataframe will not have the years as indeces as requested, but it will have the "implicit indeces" as the explicit ones

# RQ4

## RQ4.1: You should be sure there are no eponymous (different authors who have precisely the same name) in the author's dataset. Is it true?**

In [ ]:
#removing all non alphabetic characters from the beginnin & end of the string
df_ep = df_1
pattern = r'([\W_0-9\-]+$)|(^[\W_0-9\-]+)'
df_ep['plainname'] = df_ep.name.apply(lambda row: re.sub(pattern, '', row)) 

#removing all digits 
pattern = r'[0-9]|\W^ '
df_ep['plainname'] = df_ep['plainname'].apply(lambda row: re.sub(pattern, '', row)) 

#dropping rows with non valid names
df_ep = df_ep[df_ep.plainname != '']

#converting the string in lower characters only & splitting the name ([name, surname] & similia)
df_ep['plainname'] = df_ep.plainname.str.lower().str.split()

#sorting the name in order to get as eponymous the names with inverted name 6 surname order
df_ep['plainname'] = df_ep.plainname.apply(sorted)
df_ep['plainname'] = df_ep.plainname.apply(lambda row: ' '.join(row))

#Results
print("There are ", df_ep['plainname'].duplicated().sum(), "eponymous in the dataset")

If we want to get a Data Frame free from eponymous, we can run the following code. 

In [ ]:
#getting a dataframe free from eponymous
df_no_ep = df_ep.drop_duplicates(subset='plainname')

## RQ4.2: Write a function that, given a list of author_id, outputs a dictionary where each author_id is a key, and the related value is a list with the names of all the books the author has written.

In [ ]:
#FUNCTION THAT CREATES A DICTIONARY WITH AUTHORS_ID AS A KEY AND HIS/HERS BOOKS AS VALUES
def rq4(author_id_list):

    df_filt = df_2[['author_id', 'title']]
    df_filt = df_filt[ df_filt['author_id'].isin(author_id_list) ]   #keeping only the rows with author_id in author_id_list
    df_filt = df_filt.groupby(df_filt['author_id'], group_keys=False)['title'].agg(list)  
    
    d = df_filt.to_dict()
    return d

## RQ4.3: What is the longest book title among the books of the top 20 authors regarding their average rating? Is it the longest book title overall?

In [ ]:
# LONGEST BOOK TITLE AMONG THE BOOKS OF THE TOP 20 AUTHORS REGARDING THEIR AVERAGE RATING

#sorting the authors in descending order & saving only the first 20 rows of the needed data 
df_top_ratings = df_1[['id', 'average_rating']].sort_values(by= 'average_rating', ascending=False, ignore_index=True).iloc[:21]

author_id_list = list(df_top_ratings['id'])        #id in df_1 (authors) correponds to author_id in df_2 (books)
dict_authors_titles = rq4(author_id_list)

#getting all the titles of books written by the 20 top authors + flattening it 
titles = list(dict_authors_titles.values())

titles_flat = []
for aut_titles in titles:
    titles_flat += aut_titles


#getting longest title & its length (among 20 top authors' books)
titles_len_top20 = [len(title) for title in titles_flat]
max_titles_len_top20 = max(titles_len_top20)
longest_title_top20 = titles_flat[titles_len_top20.index(max_titles_len_top20)]

print("The longest title among the books written by the 20 top authors is " + longest_title_top20)


#IS IT THE LONGEST TITLE AMONG ALL THE BOOKS?

#getting the longest title among all the books 
df_2['title_len'] = df_2.title.apply(lambda row:len(row))

longest_title_len_all = df_2.title_len.max()
index_longest_title = df_2.title_len.idxmax()
longest_title = df_2['title'].loc[index_longest_title]

if longest_title_top20 == longest_title_len_all:
    if longest_title == longest_title_top20: 
        print("The longest title among all the books' ones is the same as the longest among the books of the 20 top authors ")
    else: 
        print("The  two titles selected by the algorithm are not the same, but they have the same length")
else: 
    print('They are not the same, the longest title among all books is "' + longest_title +'"')


For the second part of the question, we notice that the title is actually incomplete, and looking at the dataset, we can see that there are come other titles in the same situation.    
We think that this is due to a limit in the numbers of characters accetable to write the tilte of the book (255 characters). To address this problem, we provide a code that filters the dataset excluding such titles and returns the longet title among the complete ones. 

In [ ]:
#getting the complete longest title among all the books 
df_2['title_len'] = df_2.title.apply(lambda row:len(row))
df_longest_title = df_2[df_2.title_len < 255]

longest_title_len_all = df_longest_title.title_len.max()
index_longest_title = df_longest_title.title_len.idxmax()
longest_title = df_longest_title['title'].loc[index_longest_title]

print('The longest "complete" title among all books is:"' + longest_title+ '"' )

## RQ4.4: What is the shortest overall book title in the dataset? If you find something strange, provide a comment on what happened and an alternative answer.


In [ ]:
#SHORTEST TITLE 
shortest_title_len = df_2.title_len.min()
index_shortest_title = df_2.title_len.idxmin()
shortest_title = df_2.title.loc[index_shortest_title]

print("The shortest title along all the titles is " + shortest_title + " and its length is " + str(shortest_title_len) )

The found shortest title is one of a book that doesn't exist on GoodReads, so now we try to find the "meaningful" shortest title.

In [ ]:
#getting a title free from spaces, in order to drop all the empty & non-significant ones
df_2['no_space_titles'] = df_2['title'].apply(lambda title: title.replace(" ", ""))
df_2['no_space_title_len'] = df_2.no_space_titles.apply(lambda row:len(row))

# if we keep a lower threshold on the title's length, we get back books that don't exist on GoodReads
df_2_meaningful = df_2[ df_2['no_space_title_len'] > 3] 

#looking for the "meaningful" shortest title
index_shortest_title_meaningful = df_2_meaningful.no_space_title_len.idxmin()
shortest_title_len_meaningful = df_2.title_len.loc[index_shortest_title_meaningful] #using df_2 in order to get back the correct values
shortest_title_meaningful = df_2.title.loc[index_shortest_title_meaningful]

#Displaying results
print('The "meaningful" shortest title among all the titles is "' + shortest_title_meaningful + '" and its length is ' + str(shortest_title_len_meaningful) )

In [ ]:
#dropping columns we don't need out of this RQ
df_2.drop(['title_len', 'no_space_titles', 'no_space_title_len'] , axis= 1, inplace= True)

# RQ5

## RQ5.1 Plot the top 10 most influential authors regarding their fan count and number of books. Who is the most influential author?

In [ ]:
df_rq5_1 = df_1[['name', 'works_count', 'fans_count', 'gender']]
df_rq5_1 = df_rq5_1[df_rq5_1.name != 'Wikipedia']
df_rq5_1_grouped = df_rq5_1.groupby('name')[[ 'fans_count']].sum()
df_rq5_1_grouped = df_rq5_1_grouped.reset_index()
df_rq5_grouped_sorted = df_rq5_1_grouped.sort_values(by=['fans_count'], ascending = False)
df_rq5_grouped_sorted.head(10)

The most influential author seems to be Stephen King with 766.035 as fans_count, that nearly doubles the second, Veronica Roth who has 455.358 as fans_count

## RQ5.2 Have they published any series of books? If any, extract the longest series name among these authors

In [ ]:
influential_authors = ['Stephen King', 'Veronica Roth', 'Bill Gates', 'Colleen Hoover',	'Rick Riordan', 'James Patterson', 'John Green',
                        'Neil deGrasse Tyson', 'Cassandra Clare', 'Mindy Kaling']

df_influential_authors = df_2[df_2['author_name'].isin(influential_authors)]
df_influential_authors = df_influential_authors[df_influential_authors.series_name != '']
unique_series_names = df_influential_authors['series_name'].unique()
for series_name in unique_series_names:
    print(series_name)
longest_series_name = max(unique_series_names, key=len)

print("The longest series name is:", longest_series_name)

## RQ5.3 How many of these authors have been published in different formats? Provide a meaningful chart on the distribution of the formats and comment on it

In [ ]:
influential_authors = ['Stephen King', 'Veronica Roth', 'Bill Gates', 'Colleen Hoover',	'Rick Riordan', 'James Patterson', 'John Green',
                        'Neil deGrasse Tyson', 'Cassandra Clare', 'Mindy Kaling']

df_influential_authors = df_2[df_2['author_name'].isin(influential_authors)]
df_influential_authors = df_influential_authors[df_influential_authors.format != '']
author_format_counts = df_influential_authors.groupby('author_name')['format'].nunique()
authors_with_multiple_formats = author_format_counts[author_format_counts > 1]
print("Authors who have published in more than one format:")
print(authors_with_multiple_formats)
format_counts = df_influential_authors['format'].value_counts()
minimum_percentage = 2
different_formats = format_counts[format_counts / format_counts.sum() * 100 >= minimum_percentage]

if not different_formats.empty:
    plt.figure(figsize=(6, 6))
    plt.pie(different_formats, labels=different_formats.index, autopct='%1.1f%%')
    plt.title("Distribution of Book Formats")
    plt.show()
else:
    print("No formats meet the specified threshold for inclusion in the pie chart.")

## RQ5.4 Provide information about the general response from readers (number of fans, average rating, number of reviews, etc.), divide the authors by gender

In [ ]:
#the infromation we want to have for each author are: the number of fans, works-count, ratings_count, of average_ratings, text_reviews_count
df_rq5_4 = df_1
new_df = df_rq5_4.groupby('name').agg({
    'fans_count': 'sum',
    'works_count': 'sum',
    'ratings_count': 'sum',
    'average_rating': 'mean',
    'text_reviews_count': 'sum'
}).reset_index()
new_df = new_df.sort_values(by = 'fans_count', ascending = False)
new_df.head(10)

In [ ]:
gender_count = df_1['gender'].value_counts()
print(gender_count.head(20))

In [ ]:
other_gender = ['nonbinary', 'genderqueer', 'agender', 'genderfluid', 'gender fluid', 'agender', 'neutral', 'transgender']
df_rq5_4['gender'] = df_rq5_4['gender'].str.strip()
df_rq5_4['gender'] = df_rq5_4['gender'].str.lower()
df_rq5_4['gender'] = df_rq5_4['gender'].str.replace('-', '')
df_rq5_4 = df_rq5_4[df_rq5_4.gender != '']

def categorize_gender(x):
    if 'trans' in x:
        return 'other'
    if 'fluid' in x:
        return 'other'
    if 'they' in x:
        return 'other'
    if x in other_gender:
        return 'other'
    else:
        return x
df_rq5_4['gender'] = df_rq5_4['gender'].apply(categorize_gender)
df_rq5_4 = df_rq5_4[df_rq5_4['gender'].isin(['male','female','other'])]
a = df_rq5_4['gender'].value_counts()
print(a)

gender_counts = df_rq5_4['gender'].value_counts()
plt.figure(figsize=(6, 6))
plt.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Gender Distribution of authors')
plt.show()


By looking at the data under the column 'gender' of df_1, we noticed that many genders, different from 'male' and 'female', had strange values, not linked with the genders different from the mian 2. After this discovery we decided to create a new category called 'other' where real genders, that are not male and female could be placed in.
Analyzing the final pie chart about the distribution of genders, we can clearly see how the main 2 genders are the most diffused among the book's authors; at the same time, we would like to underline the not complete absence of other genders, that could be seen as an intial step for a future increase of the number of authors that identify in one of the categories inside 'other'.

Talking about structural biases, they can manifest themselves in various ways, but in this case the overrepresentation of male authors over female authors seems absent; in fact, the 2 percentages are not so far from each other and this doesn't create unbalancement.
Having an umbalanced dataset, in this case, could limit the diversity of voices and perspectives in literature, leading to a narrower range of experiences and viewpoints in the books that are published. It's important to consider the historical and social factors that contribute to this topic. Recognizing and addressing structural bias in literature and authorship is crucial for promoting diversity and inclusivity, is essential to create an environment where authors of all genders have equal opportunities to share their stories and experiences.

# RQ6

## RQ6.1 Provide the average time gap between two subsequent publications for a series of books and those not belonging to a series. What do you expect to see, and what is the actual answer to this question?

In [ ]:
# we need to split the books that belong to a series and the books that doesn't belong to a series, we assume that the books with empty values in the columns 'series_name' and 'series_id', don't belong to a series, so are single books
book_series = df_influential_authors[df_influential_authors['series_name'] != '']
book_series = df_influential_authors[df_influential_authors['series_id'] != '']

In [ ]:
#we drop all the cells where there isn't the publication date because if there isn't we can't claculate the difference: publication_date - original_publication_date = time gap
bs_without_missing = book_series[book_series['publication_date'] != '']
bs_without_missing

In [ ]:
bs_without_missing['publication_date'] = pd.to_datetime(bs_without_missing['publication_date'], format='%Y', errors='coerce')
bs_without_missing['original_publication_date'] = pd.to_datetime(bs_without_missing['original_publication_date'], format='%Y', errors='coerce')
bs_without_missing['publication_difference_years'] = bs_without_missing['publication_date'].dt.year - bs_without_missing['original_publication_date'].dt.year
mask = (bs_without_missing['publication_date'].dt.year <= 2023) | (bs_without_missing['publication_date'].isnull())
bs_without_missing = bs_without_missing.loc[mask].dropna()
average_difference = bs_without_missing['publication_difference_years'].mean()
print("Average Difference in Years:", average_difference)

In [ ]:
book_without_series = df_influential_authors[df_influential_authors['series_name'] == '']
book_without_series = df_influential_authors[df_influential_authors['series_id'] == '']

In [ ]:
bsws_without_missing = book_without_series[book_without_series['publication_date'] != '']
bsws_without_missing

In [ ]:
bsws_without_missing['publication_date'] = pd.to_datetime(bsws_without_missing['publication_date'], format='%Y', errors='coerce')
bsws_without_missing['original_publication_date'] = pd.to_datetime(bsws_without_missing['original_publication_date'], format='%Y', errors='coerce')
bsws_without_missing['publication_difference_years'] = bsws_without_missing['publication_date'].dt.year - bsws_without_missing['original_publication_date'].dt.year
mask = (bsws_without_missing['publication_date'].dt.year <= 2023) | (bsws_without_missing['publication_date'].isnull())
bsws_without_missing = bsws_without_missing.loc[mask].dropna()
bsws_average_difference = bsws_without_missing['publication_difference_years'].mean()
print("Average Difference in Years:", bsws_average_difference)

Before plotting the results the assumpion we made was that the gap should have been more or less the same, basing our hypothesis on the fact that what really influences the authors' productivity isn't the belonging of a book to a seires or not, but other factors such as: the time management, inspiration and creativity, personal experiences, motivation or the emotional state. Related to the previous statement, we would like to underline how the market logics, where the publishers could stress an author to realease a new book consistenly, isn't really related to a book belonging to a series or not, but is related to how the market percieves that author, how many fans she/he has and what is the degree of influence on the market.

## RQ6.2 For each of the authors, give a convenient plot showing how many books has the given author published UP TO a given year. Are these authors contemporary with each other? Can you notice a range of years where their production rate was higher?

In [ ]:
df_influential_authors['publication_date'] = pd.to_datetime(df_influential_authors['publication_date'], format='%Y', errors='coerce')
df_influential_authors['publication_year'] = df_influential_authors['publication_date'].dt.year
author_year_counts = df_influential_authors.groupby(['author_name', 'publication_year']).size().reset_index(name='work_count')
author_year_counts['cumulative_books'] = author_year_counts.groupby('author_name')['work_count'].cumsum()
target_year = 2022
data_for_target_year = author_year_counts[author_year_counts['publication_year'] <= target_year]

for author, data in data_for_target_year.groupby('author_name'):
    plt.plot(data['publication_year'], data['cumulative_books'], label=author)
plt.xlabel('Year')
plt.ylabel('Cumulative Number of Books Published')
plt.yscale("log")
plt.title(f'Books Published by Authors up to {target_year}')
plt.legend(loc='best')

plt.show()

The plots shows how these authors are conteporary, with just Stephen King and James Patterson having the first books published between the 70s and the 80s. All the other authors started to published their books in the first years of the 2000s, in fact, the lines of the overlap around that years.

# RQ7 

## RQ7.1: Estimate the probability that a book has over 30% of the ratings above 4.

In [ ]:
#organizing data
df_ratings = df_2[['work_id', 'rating_dist', 'ratings_count']]

#initilizing the number of ratings over 4 (which are the 5 stars ratings) overwriting on rating_dist
df_ratings.loc[:,'rating_dist'] = df_ratings['rating_dist'].apply(lambda row: row[4] + row[5])

#dividing books between having or not 30% of ratings above 4
df_ratings.loc[:, '30_over_4'] = df_ratings.apply(lambda row: bool( row['rating_dist']/row['ratings_count'] > 0.3), axis= 1)

#probability estimation
books_30over4 = sum(df_ratings['30_over_4'])
books_total = len(df_ratings)

probability_30over4 = books_30over4/books_total

#Showing results
print("The probability that a book has over 30 percent of ratings over 4 is:", round(probability_30over4, 2))

## RQ7.2: Estimate the probability that an author publishes a new book within two years from its last work.

In [ ]:
#getting relevant data
df_publications = df_2[['author_id', 'work_id', 'original_publication_date']]
df_publications.drop_duplicates(subset='work_id', inplace=True)

#pre-processing 'publication_date' column 
pattern = r'(^[0-9]{4}-[0-9]{2}-[0-9]{1,2}$)|(^[0-9]{4}$)|(^[0-9]{4}-[0-9]{1,2}$)'
df_publications= df_publications[df_publications['original_publication_date'].apply(lambda row: bool(re.search(pattern, row)))]
df_publications['original_publication_date'] = df_publications['original_publication_date'].apply(lambda row: re.split('-', row)) #this way we obtain year=df_rq3['original_publication_date'][0], month=df_rq3['original_publication_date'][1]
df_publications['original_publication_year'] = df_publications['original_publication_date'].apply(lambda row: int(row[0]))

#grouping
df_publications = df_publications.groupby('author_id').agg(list)

#getting the number of authors that published books in less than two years apart
df_publications['original_publication_year'] = df_publications.original_publication_year.apply(lambda row: sorted(row)[::-1] )  #sorting publication years in descending order
df_publications['years_diff'] = df_publications.original_publication_year.apply(lambda row: [ ( (row[i] - row[i+1]) <= 2 ) for i in range(len(row)-1)])
closerthan2years=[1 for row in df_publications['years_diff'] if sum(row)>0]

#estimating probability
probability_closer_than2years = sum(closerthan2years)/len(df_publications)
print("The probability that an author publishes a new book  within two years from its last work is:", round(probability_closer_than2years, 2))

## RQ7.3: In the file list.json, you will find a peculiar list named "The Worst Books of All Time." Estimate the probability of a book being included in this list, knowing it has more than 700 pages.

In [ ]:
#loading data
#to load lists.json, set to_be_loaded == True
to_be_loaded = True

if to_be_loaded == True:
    lists = pd.read_json('list.json', lines=True)

#first step: getting needed data
    # index of "The worst books of all time" == 0
worst_of_all_time = lists.books.iloc[0]    #it's a list
total_books_worst = int(lists.num_books.iloc[0])


#getting a list of book_keys of the books in "The worst books of all time"
ids = []
for book in worst_of_all_time: 
    ids.append(int(book['book_id']))

#number of books with more than 700 pages in "The worst books of all time"
num_pages_list = df_2['num_pages'][df_2['id'].isin(ids)]
more_than_700_worst = sum([ (pages>700) for pages in num_pages_list ])

#number of books with more than 700 pages in all the dataset
more_than_700_all = len(df_2[df_2['num_pages'] > 700]) 

#conditional probability 
probability_worst_over700 = more_than_700_worst / more_than_700_all

#show results
print('The probability that a book is in "The worst books of all time" knowing that it has more than 700 pages is:', round(probability_worst_over700, 2))


## RQ7.4: Are the events X=’Being Included in The Worst Books of All Time list’ and Y=’Having more than 700 pages’ independent? Explain how you have obtained your answer.

We check if the events *X=’Being Included in The Worst Books of All Time list’* and *Y=’Having more than 700 pages’* are independent by comparing *Pr(X | Y)* obtained in the prevous step with *P(X)*. Indeed, we know that these two values are the same iff the two events are independent. \
We keep track of the possible numeric error raised by the use of a computer using *math.isclose* instead of *==*.

In [ ]:
#ARE X = ’Being Included in The Worst Books of All Time list’ AND Y =’Having more than 700 pages’ INDEPENDENT? 

probability_inworst = total_books_worst/len(df_2)
if math.isclose(probability_worst_over700, probability_inworst):
    print("The two evwnts are independent.")
else:
    print("The two evwnts aren't independent.")

# RQ 8 

## RQ8.1: Can you demonstrate that readers usually rate the longest books as the worst?


In [ ]:
#1st procedure: correlation coefficient

df_2[['average_rating', 'num_pages']][(df_2['average_rating']>=1)].corr()

In [ ]:
#2nd procedure: linear regression + visualization

''' X = number of pages
    Y = average rating '''

X = df_2.num_pages[(df_2['average_rating']>=1)]
Y = df_2.average_rating[(df_2['average_rating']>=1)]
Z=X[X<20000]

plt.scatter(X[(Y>0) & (X<20000)], Y[(Y>0) & (X<20000)], 1)

X = sm.add_constant(X, prepend=True, has_constant='skip')   #adding constant to get the intercept (not given in default mode in ols)

regression_model = sm.OLS(Y, X)
regression_results = regression_model.fit()

b1 = regression_results.params['num_pages']
b0 = regression_results.params['const']
print("Regression coefficient:", b1)


plt.plot(Z, b0 + b1*Z, color= 'red') # Y = b0 + b1*X is the equation of the "regession line"

plt.xlabel('Number of pages')
plt.ylabel('Average Rating')

plt.title('Regression plot over scatterplot of the dataset')

plt.show()

For this task, we chose to consider only the books with average rating greater or equal to $1$ because the ratings' distribution takes values from 1 to 5, so those books are the ones which don't have any ratings or have misrecorded avarege.

To address this question, we followed two methodologies: the first, and faster, was to look at the correlation coefficient between the books' avarege rate and number of pages. 
As showed above, the correlation coefficient we got is reaaly close to zero, meaning that there is no relevant correlation between the variables we took into exam. 
Moreover, we choose to also perform a linear regression in order to corroborate our results. This procedure is also useful to get a visual representation of our analysis.
The risults of this second procedure confirmed the ones given from the correlation coefficient. Indeed, we got a regression coefficient proportional to $10^{-10}$, which is practically zero, meaning that the two variables are orthogonal and not corelated. 
On a sidenote, to make the plot of our results actually readable, we choose to plot the books with less than $20000$  pages.


## RQ8.2: Compare the average rate distribution for English and non-English books with a proper statistical procedure. What can you conclude about those two groups?

In [ ]:
#getting needed data
english_books = df_2[df_2['language'].isin(['eng', 'en-US', 'en-GB', 'US'])]['average_rating']
non_english_books = df_2[~df_2['language'].isin(['eng', 'en-US', 'en-GB', 'US',  ''])]['average_rating'] #excluding also the books that don't have the language registered, cause they could be misclassified if keeped

#excluding books with average rating <1 because misregistered
english_books = english_books[english_books >= 1 ]
non_english_books = non_english_books[non_english_books >= 1]

In [ ]:
numbins=50

#english book average rate
plt.subplot(1, 2, 1)
hist_en, bins_en, _ = plt.hist(english_books, bins=numbins, density=True, alpha=0.5, color='b', edgecolor='k')

# highlighting mean and median
mean_en = np.mean(english_books)
median_en = np.median(english_books)
plt.axvline(mean_en, color='purple', linestyle='dotted', linewidth=2, label='Mean')
plt.axvline(median_en, color='r', linestyle='dotted', linewidth=2, label='Median')


plt.grid(axis= 'y', linestyle='dotted')
plt.xlabel('Average rate')
plt.title("English books")
plt.legend()

#non english books average rate
plt.subplot(1,2,2)
hist_non_en, bins_non_en, _ = plt.hist(non_english_books, bins=numbins, density=True, alpha=0.5, color='b', edgecolor='k')

# highlighting mean and median
mean_nonen = np.mean(non_english_books)
median_nonen = np.median(non_english_books)

plt.axvline(mean_nonen, color='purple', linestyle='dotted', linewidth=2, label='Mean')
plt.axvline(median_nonen, color='r', linestyle='dotted', linewidth=2, label='Median')

plt.grid(axis= 'y', linestyle='dotted')
plt.xlabel('Average rate')
plt.title("Non-english books")
plt.legend()


plt.tight_layout()
plt.show()

From a first visual exploration of the two groups of the dataset we don't notice any valuable difference between them. 

In [ ]:
#descriptions
print("English books statistics: \n") 
english_books.describe()

In [ ]:
print("Non-english books statistics: \n")
non_english_books.describe()

From the meaningful statistics plotted above we notice that the English-books group is larger than the other one, but their relevant statistics aren't that different. 

**OPTION 1: t-Test**

Warning: we know that when performing a t-Test the data distribution is required to be Normal. We tested  the samples for that using the pre-implemented Shapiro test (using *stats.shapiro(data)* ), but we got a warning back saying the following: "*UserWarning: p-value may not be accurate for N > 5000.*"

On the other hand, we know that for i.i.d. r.v., their mean is asintotically Normal for the Central Limit Theorem. The problem is that in this case, the samples may not be identically distributed: euristically, the ratings of a book are strictly book dependet, and so is their mean. For this reasons, we choose to go for a non parameric Kolmogorov-Shmirov test, which isn't distribution-dependent.

**OPTION 2: Kolmogorov-Shmirov test**

In [ ]:
#The null hypothesis is that the two distributions are identical, F(x)=G(x) for all x;
from scipy import stats
ks_stat, p_value = stats.ks_2samp(english_books, non_english_books)

# Results 
alpha = 0.05  #level of significance

if p_value < alpha:
    print(f"The two samples are statistically different (p = {p_value})")
else:
    print(f"The two samples are not statistically different (p = {p_value})")

Since the samples are different, we check which one is dominant.

In [ ]:
# alternative='greater': The null hypothesis is that F(x) <= G(x) for all x

ks_stat_great, p_value_great = stats.ks_2samp(english_books, non_english_books, alternative='greater')

# Results 
if p_value_great< alpha:
    print(f"The distribution of english books is stocastically dominant (p = {p_value_great})")
else:
    print(f"The distribution of non-english books is stocastically dominant  (p = {p_value_great})")

In [ ]:
#plot to confront the distributions

datarange= max(english_books) - min(english_books)

bin_centers_en = bins_en+ (0.5*datarange/numbins)           #numbins, bincenters took from the previous plot(histograms)
bin_centers_non_en = bins_non_en+ (0.5*datarange/numbins)


plt.subplot(1,1,1)
plt.plot(bin_centers_en[:-1], hist_en, 'r-', linewidth=1, label='English')
plt.plot(bin_centers_non_en[:-1], hist_non_en, 'g-', linewidth=1, label='Non-English')

plt.title("English and Non-English books' distributions")
plt.xlabel('Average Rate')
plt.ylabel('')
plt.legend()
plt.show()

## RQ8.3: About the two groups in the previous question, extract helpful statistics like mode, mean, median, and quartiles, explaining their role in a box plot.

In [ ]:
plt.subplot(1,2,1)
plt.boxplot(english_books, meanline=True, showmeans=True)
plt.grid(True, linestyle='dotted')
plt.title("English books")


plt.subplot(1,2,2)
plt.boxplot(non_english_books, meanline=True, showmeans=True)
plt.grid(True, linestyle='dotted')
plt.title("Non-english books")


plt.show()
print("Some meaningful statistics about english books' average rate are: \n", non_english_books.describe())
print("Some meaningful statistics about non-english books' average rate are: \n", non_english_books.describe())


## RQ8.4: It seems reasonable to assume that authors with more fans should have more reviews, but maybe their fans are a bit lazy. Confirm or reject this with a convenient statistical test or a predictive model.

To perform this task, we decided to use the informations in *"ratings_count"* and not the ones in *"text_reviews_count"* because we interpreted also the ratings as a review of the book, in a broader sense.

In [ ]:
#Linear regression to estimate the number of reviews based on fan numbers and seeing if the "regression line" (come si chiama?) has a positive coefficient 

#X = fans count;  Y =  ratings count.
# Y = b0 + b1 X

X = df_1.fans_count
Y = df_1.ratings_count
X = sm.add_constant(X, prepend=True, has_constant='skip')   #adding constant to get the intercept (not given in default mode in ols)

regression_model = sm.OLS(Y, X)
regression_results = regression_model.fit()

#getting regression coefficients
b1 = regression_results.params['fans_count']
b0 = regression_results.params['const']

print("Regression coefficient b1:", b1)
print("Regression coefficient b0 (intercept):", b0)

#F test
f_statistic = regression_results.fvalue
p_value = regression_results.f_pvalue
print("F-statistic:", f_statistic)
print("F - test p-value:", p_value)

#rsquared
print('R-squared:', regression_results.rsquared) 

In [ ]:
#plot regression vs points 

X = df_1.fans_count
Y = df_1.ratings_count

plt.scatter(X, Y, 1)
plt.plot(X, b0 + b1*X, color= 'red') #b0 + b1*X is the ewuation of the "regession line"

plt.xlabel('Fans count')
plt.ylabel('Ratings count')

plt.title('Regression plot over scatterplot of the dataset')

plt.show()


To answer this question, we decided to perform a linear regression procedure against the two analized variables: "fans_count" and "ratings_count".

The results show that the model isn't actually very explicaytive (we have a low Rsquared coefficient), but on the other hand the assumption it's true for the well-represented part of the data. 
On the other hand, the F-test results tell us that our model has a better fit than the one with only the intercept.
In the end, we have that the regression coefficient we obtained is positive, meaning that there's a direct proportionality relationship between the two analized variables, so the assumption made in this case (the fact that the authors with more fans should haeìve more ratings) is true.

To corroborate this affermation, we decided to also calculate the correlation coefficient between the variables. 

In [ ]:
df_1[['fans_count', 'ratings_count']].corr()

The correlation coefficient we got is approximatly $0.6$, meaning that the two variables aren't orthogonal/independent, but indeed there's a positive correlation btween them, which leads to a type of relationship like "when one grows, so does the other".

**8.5: Provide a short survey about helpful statistical tests in data analysis and mining: focus on hypothesis design and the difference between parametric and nonparametric tests, explaining the reasons behind the choice of one of these two tests.**

Statistical tests are useful to get information about data distribution and other statistical attributes. In particoular, they could be used to confirm or reject assumption about data. 
In both cases, the data must themselves satisfy assumptions in order for the tests to be correctly applicable. 
In the case of parametric tests, normality of the data (i.e. that their distribution is normal), assumptions about the variance of the data, and independence of the observations are often required. 
Obviously, these characteristics constitute a discriminating factor in the choice of test to be applied, along with the type of analysis that is required.
In the event that the data do not verify the required assumptions on the distribution, but only that of independence of the observations, it is plausible that a non-parametric test is applicable, which is independent of the distribution of the data.

Hypothesis tests can also be divided into bilateral and unilateral tests. This distinction refers to the type of hypotheses that are tested, i.e. how these hypotheses are designed. In the case where, under the assumption that the null hypothesis is one of equality (between two parameters in question), the alternative hypothesis is one of inequality ($\neq$), we speak of bilateral tests, whereas in the case where the alternative is one of strict inequality ( $>$, $<$) we speak of unilateral tests. The difference refers to the tails of the distribution that are considered in the analysis.

Some useful tests in this type of analysis are: 
- t-test: parametric test used to check whether the mean value of a distribution deviates significantly from a fixed value or not. The test is based on Student's t-distribution, requires the assumption of normality and is based on the following statistic: 
\begin{equation*}
Y_{0}={\frac {\bar{X} -\mu _{0}}{s/{ \sqrt{n}}}}
\end{equation*}
Knowledge of the variance of the distribution is not required.

- F-test: is a parametric test to verify that two normal distributions have the same variance. It is based on Fisher's distribution and requires sample independence. The reference statistic for such a test is: 
\begin{equation*}
F={\frac {S_{X}^{2}}{S_{Y}^{2}}}
\end{equation*}, 
where $S_{X}$ and $S_{Y}$ are estimators of the variance of the two distributions $X$ and $Y$.
It is verified that $F$ follows the Fisher distribution under the assumption $H_{0}=(\sigma _{X}^{2}=\sigma _{Y}^{2})$, hence the name of the test.

- Kolmogorov-Smirnov test: is a non-parametric test aimed at comparing either the distribution of a sample with a known distribution, or the distributions of two samples. It requires the independence of the samples in the two-sample case; in the two-sample case, their empirical distribution is used. The assumptions tested in the two-sample case are as follows: 
\begin{align*}
& H_{0}: {F(x)=F_{0}(x), \forall x} \\
& H_{1}: {\exists x : F(x)\neq F_{0}(x)}
\end{align*}
With $F$, $F_0$ the two distributions under consideration. The test statistic is the sup distance between the two (possibly empirical) distributions.

# Bonus Points - Question 1

## Bonus points 1.1: Select one alternative library to Pandas (i.e., Dask, Polar, Vaex, Datatable, etc.), upload authors.json dataset, and filter authors with at least 100 reviews. Do the same using Pandas and compare performance in terms of milliseconds.

### Using Pyspark

I first installed PySpark by:
$ pip install pyspark

- Then, used 'SparkSession.builder.appName("FilterAuthors").getOrCreate()' to create a Spark session in order to use its built-in libraries and methods.
- I read the json file with '.read.json()' method. PySpark is basically designed to work with Big data.
- Pyspark uses resilient distributed datasets (RDDs) to work parallel on the data. Hence, it performs better than pandas.
- PySpark uses lazy processing, retrieving data from disk only when required, while the pandas module stores all data in memory, resulting in higher memory consumption compared to PySpark.

In [ ]:
%%time

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create a Spark session
spark = SparkSession.builder.appName("FilterAuthors").getOrCreate()

# Read the JSON file
df = spark.read.json("lighter_authors.json")

# Filter the DataFrame based on the text_reviews_count property
filtered_df = df.filter(col("text_reviews_count") >= 100)

# Sort the DataFrame in descending order by 'text_reviews_count'
sorted_df = filtered_df.sort(col("text_reviews_count").desc())

# Select specific columns and return the first 5 rows
result_df = sorted_df.select("id", "name", "text_reviews_count").limit(5)

# Show the resulting DataFrame
result_df.show()

# Stop the Spark session
spark.stop()

+-------+---------------+------------------+
|     id|           name|text_reviews_count|
+-------+---------------+------------------+
|   3389|   Stephen King|            608956|
|1077326|   J.K. Rowling|            606373|
| 153394|Suzanne Collins|            427224|
| 150038|Cassandra Clare|            416177|
|3433047|  Sarah J. Maas|            372923|
+-------+---------------+------------------+

CPU times: user 38.1 ms, sys: 11.3 ms, total: 49.5 ms
Wall time: 2.61 s


### Using Pandas

In [ ]:
%%time
import pandas as pd

authors_json = pd.read_json('./lighter_authors.json', lines = True, chunksize=100)

authors_df = pd.concat(authors_json, ignore_index=False)

# Filter the DataFrame based on 'text_reviews_count'
filtered_df = authors_df[authors_df['text_reviews_count'] >= 100]

# Sort the filtered DataFrame in descending order by 'text_reviews_count'
sorted_df = filtered_df.sort_values(by='text_reviews_count', ascending=False)

# Select specific columns and return the first 5 rows
result_df = sorted_df[['id', 'name', 'text_reviews_count']].head(5)

# Print the resulting DataFrame
print(result_df)


             id             name  text_reviews_count
1017       3389     Stephen King              608956
86500   1077326     J.K. Rowling              606373
27522    153394  Suzanne Collins              427224
27110    150038  Cassandra Clare              416177
157593  3433047    Sarah J. Maas              372923
CPU times: user 23.2 s, sys: 1.93 s, total: 25.1 s
Wall time: 25.1 s


### Compare the performance of PySpark Vs Pandas:

| Library     | CPU user    | sys         | total       | Wall time   | 
| ----------- | ----------- | ----------- | ----------- | ----------- |
| PySpark | 38.1 ms | 11.3 ms | 49.5 | 2.61 |
| Pandas | 23.2 s | 1.93  | 25.1 s | 25.1 s |

## Bonus points 1.2: Select one alternative library to Pandas (i.e., Dask, Polar, Vaex, Datatable, etc.), upload books.json, and join them with authors.json based on author_id. How many books don’t have a match for the author?

### First approach

In [ ]:
# Create a Spark session
spark = SparkSession.builder.appName("JoinBooksAndAuthors").getOrCreate()

# Read the 'lighter_books.json' and 'lighter_authors.json' files
books_df = spark.read.json("lighter_books.json")
authors_df = spark.read.json("lighter_authors.json")

In [ ]:
# Get the column names for each DataFrame for better understanding of the structure of datasets.
books_columns = books_df.columns
authors_columns = authors_df.columns

# Print the column names
print("Columns in 'lighter_books.json':", books_columns)
print("------------------------------------------------------------")
print("Columns in 'lighter_authors.json':", authors_columns)


Columns in 'lighter_books.json': ['asin', 'author_id', 'author_name', 'authors', 'average_rating', 'description', 'edition_information', 'format', 'id', 'image_url', 'isbn', 'isbn13', 'language', 'num_pages', 'original_publication_date', 'publication_date', 'publisher', 'rating_dist', 'ratings_count', 'series_id', 'series_name', 'series_position', 'shelves', 'text_reviews_count', 'title', 'work_id']
------------------------------------------------------------
Columns in 'lighter_authors.json': ['about', 'average_rating', 'book_ids', 'fans_count', 'gender', 'id', 'image_url', 'name', 'ratings_count', 'text_reviews_count', 'work_ids', 'works_count']


In [ ]:
# Perform the join operation. books with the same author_id as authors with the same id, will be joined together.   
joined_df = books_df.join(authors_df, books_df['author_id'] == authors_df['id'], 'left_outer')

# Count the number of books without a matching author
books_without_author = joined_df.filter(authors_df['id'].isNull()).count()

# Count the number of books that couldn't be joined
unjoined_count = joined_df.filter(authors_df['id'].isNull()).count()

# Show the count
print(f"Number of books that could not be joined: {unjoined_count}")

# Stop the Spark session
spark.stop()

Number of books without a matching author: 0


### Second approach

In this approach, 
- first, we return the number of rows in lighter_books.json. which is stored in 'num_rows_books'.
- Then, we define a variable named 'joined_df_count' as an incrementation. if any row of lighter_books.json and any row of lighter_authors.json were be joint, increase 'joined_df_count' for 1. or basically calculate the count of joined_df.
- Then, we return the difference between the 'joined_df_count' and number of rows from the books.json file.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create a Spark session
spark = SparkSession.builder.appName("JoinAndCount").getOrCreate()

# Read 'lighter_books.json' and 'lighter_authors.json'
books_df = spark.read.json("lighter_books.json")
authors_df = spark.read.json("lighter_authors.json")

# Join the two DataFrames based on 'author_id' and 'id'
joined_df = books_df.join(authors_df, books_df['author_id'] == authors_df['id'], 'inner')

# Calculate the number of rows in 'lighter_books.json'
num_rows_books = books_df.count()

# Define and increment the variable 'k' for each matching row
joined_df_count = joined_df.count()

# Calculate the difference between 'k' and the number of rows in 'lighter_books.json'
difference = joined_df_count - num_rows_books

# Print the result
print("Number of Rows in lighter_books.json:", num_rows_books)
print("Value of 'joined_df_count':", joined_df_count)
print("Difference:", difference)

# Stop the Spark session
spark.stop()


23/10/31 12:26:03 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Number of Rows in lighter_books.json: 7027431
Value of 'joined_df_count': 7027431
Difference: 0


### Conclusion
As can be seen, all books have an id that exists in authors.json file

# Bonus Points 2

## Bonus points 2.1 & 2.2: 
## If you choose to text-mine books.json ’ descriptions, try to find a way to group books in genres using whatever procedure you want, highlighting words that are triggers for these choices.
## If you choose to text-mine authors.json’ about-field, try to find a way to group authors in genres using whatever procedure you want, highlighting words that are triggers for these choices.



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, udf
from pyspark.sql.types import StringType

# Create a Spark session
spark = SparkSession.builder.appName("GroupAuthorsIntoGenres").getOrCreate()

# Read 'lighter_authors.json' and 'lighter_books.json'
authors_df = spark.read.json("lighter_authors.json")
books_df = spark.read.json("lighter_books.json")

# Define triggers for genres
genre_triggers = {
    "Romance": ["love", "relationship", "heart", "passion", "kiss", "romantic", "affection"],
    "Mystery": ["mystery", "detective", "crime", "suspense", "puzzle", "enigma", "whodunit"],
    "Fantasy": ["fantasy", "magic", "kingdom", "mythical", "enchanted", "magical"],
    "Science Fiction": ["science fiction", "space", "alien", "future", "technology", "extraterrestrial"],
    "Non-Fiction": ["non-fiction", "memoir", "history", "biography", "documentary", "autobiography"],
    "Adventure": ["adventure", "quest", "journey", "exploration", "excitement", "expedition"],
    "Horror": ["horror", "scary", "fear", "terror", "supernatural", "haunting"],
    "Comedy": ["comedy", "funny", "humor", "laughter", "comic", "hilarious"],
    "Drama": ["drama", "tragedy", "emotional", "theatre", "intense", "performing"],
    "Thriller": ["thriller", "suspenseful", "intense", "nail-biting", "tension", "exciting"],
    "Biography": ["biography", "life story", "autobiography", "history", "memoir", "personal journey"],
    "Fantasy": ["fantasy", "magic", "kingdom", "mythical", "enchanted", "magical"],
    "Science Fiction": ["science fiction", "space", "alien", "future", "technology", "extraterrestrial"],
    "Historical": ["historical", "period", "past", "epoch", "history", "retro"],
    "Self-Help": ["self-help", "personal growth", "motivation", "positive", "self-improvement", "inspiration"],
    "Cooking": ["cooking", "culinary", "recipes", "food", "chef", "cuisine"],
    "Travel": ["travel", "adventure", "exploration", "journey", "destination", "vacation"],
    "Science": ["science", "scientific", "discovery", "research", "knowledge", "experiment"],
    "Children": ["children", "kids", "juvenile", "childhood", "youth", "picture book"],
    "Poetry": ["poetry", "verse", "rhyme", "lyrical", "poem", "stanza"],
}

# Define a UDF to assign genres based on triggers.
# The UDF assign_genre iterates through the genre_triggers dictionary.
# For each genre, it iterates through the list of triggers associated with that genre.
# It checks if any of these triggers (converted to lowercase for case-insensitive matching) are found in the 'about' text of the author.
# If a trigger is found in the 'about' text, the UDF returns the associated genre.
def assign_genre(text):
    for genre, triggers in genre_triggers.items():
        for trigger in triggers:
            if trigger in text.lower():
                return genre
    return None

# Register the UDF with Spark
assign_genre_udf = udf(assign_genre, StringType())

# Create a new column 'genre' based on the 'about' field
authors_df = authors_df.withColumn("genre", when(col("about").isNotNull(), assign_genre_udf(col("about"))).otherwise(None))

# Create a new column 'genre' based on the 'description' field
books_df = books_df.withColumn("genre", when(col("description").isNotNull(), assign_genre_udf(col("description"))).otherwise(None))

# Select and show the resulting DataFrame with 'id', 'name', and 'genre' columns
result_authors_df = authors_df.select("id", "name", "genre")
result_authors_df.show()

# Select and show the resulting DataFrame with 'id', 'title', and 'genre' columns
result_books_df = books_df.select("id", "title", "genre")
result_books_df.show()

# Stop the Spark session
spark.stop()


+---+--------------------+---------------+
| id|                name|          genre|
+---+--------------------+---------------+
|  4|       Douglas Adams|        Romance|
|  7|         Bill Bryson|    Non-Fiction|
| 10|         Jude Fisher|        Fantasy|
| 12|James Hamilton-Pa...|    Non-Fiction|
| 14|         Mark Watson|         Comedy|
| 16|       Edith Wharton|        Romance|
| 17|       Luther Butler|        Romance|
| 18|        Gary Paulsen|        Romance|
| 20|           Dale Peck|       Children|
| 23|       Angela Knight|        Romance|
| 24|       Delia Sherman|        Fantasy|
| 25|Patricia A. McKillip|        Romance|
| 26|      Anne McCaffrey|Science Fiction|
| 27|Zilpha Keatley Sn...|       Children|
| 29|        Kate Horsley|        Romance|
| 31|   Elaine Cunningham|        Fantasy|
| 32|       Philippa Carr|     Historical|
| 33|     Edward P. Jones|       Children|
| 36|        Satyajit Das|           NULL|
| 38|         Mark Smylie|        Fantasy|
+---+------

## Bonus points 2.3: If you feel comfortable and did both tasks, analyze the matching of the two procedures. You grouped books and authors in genres. Do these two procedures show correspondence?

As can be seen, if we increase the number of triggers for genres, the accuracy of dedicating of a genre would be higher.
- After reading two datasets using PySpart, we defined an Object with some key value pairs. the keys are the geners that we want to dedicate to each book or author's description or about part. the values are the list of words that we search for them in the description and the about text.
- then, we defined a User-Defined Function (UDF) to assign genres to authors and books based on certain "triggers" found in their 'about' and 'description' text.
- A UDF is a way to define a custom function that can be applied to the columns of a PySpark DataFrame.
- Then, We add a new column to the authors and books datafram, and wrote down the genres.
- The same approach works fine for both of the frameworks.


# ADM Homework 2 - AWS Question (AWSQ)

In [ ]:
import pandas as pd
import json
import time

In [ ]:
%%time
list_json = pd.read_json('list.json', lines = True, chunksize=10000)
list_df = pd.concat(list_json, ignore_index=False)

# Explode the 'tags' column to create separate rows for each tag
exploded_tags = list_df['tags'].explode()

# Count the frequency of each tag
tag_counts = exploded_tags.value_counts()

# Get the top 5 most frequently used tags
top_5_tags = tag_counts.head(5)

# Print the result
print(top_5_tags)

tags
romance            6001
fiction            5291
young-adult        5016
fantasy            3666
science-fiction    2779
Name: count, dtype: int64
CPU times: user 30.7 s, sys: 59.7 s, total: 1min 30s
Wall time: 2min 13s


## The information about the config of the EC2 instance
first I created a instance:
in Name and tags filed, I wrote:
linux-t3-large
in ‘Application and OS Images (Amazon Machine Image)’ I select Ubuntu Server 22.04 LTS

as instance type, I selected: 
t3.large

as Key pair, I selected my existing key that I used to connect.

In Network settings, I chose:
Select existing security group

as Storage (volumes), I select 30 Size (GiB)

Then pressed ‘Launch Instance’. 


-----------------------------------------------------------------------

## The command used to connect to the EC2

I used the following command to connect to my EC2 linux server (instance):

$ ssh -i "myKeyPair-Arman.pem" ubuntu@ec2-54-204-98-221.compute-1.amazonaws.com

First I uploaded the file in my S3 bucket.

in EC2, I ran the following command to download from S3 bucket. 
$ wget https://arman-feili.s3.amazonaws.com/list.json

In order to run Jupyter Notebook in AWS linux server, I ran the following command in linux terminal:
$ jupyter notebook --no-browser

I ran the below command in my local terminal:
$ ssh -L 8088:localhost:8888 -i "myKeyPair-Arman.pem" ubuntu@ec2-54-204-98-221.compute-1.amazonaws.com

in order to see the Jupyter notebook window in a Graphical user interface, I ran: 
http://localhost:8088

-----------------------------------------------------------------------

## The commands used to upload the files and run the script on the EC2 instance through your local system

I created a S3 bucket as an extra storage for EC2 instance.

I clicked on ‘S3’ service.
then pressed ‘create bucket’ and defined a name which was unique.

for the AWS Region, chose:
US East (N. Virginia) us-east-1

then went to permissions
> Block public access (bucket settings)
> Edit
unchecked all the blocks, so your bucket would be public.

Went to permissions > Object OwnershipInfo > Edit
change to ‘ACLs enabled’

I uploaded the list.json file into the S3
I changed the permission for Read and Write for everyone, so I could easily execute the following command to download list.json file from S3 bucket

-----------------------------------------------------------------------

## A table containing the most popular tags and their number of usage

tags
romance:            6001,
fiction:            5291,
young-adult:        5016,
fantasy:            3666,
science-fiction:    2779

### Tags:
| romance | fiction | young-adult | fantasy | science-fiction
| --- | --- | --- | --- | --- |
| 6001 | 5291 | 5016 | 3666 | 2779 |

-----------------------------------------------------------------------

## A table containing the running time of the script on your local system and EC2 instance

local machine:
CPU times: user 30.7 s, sys: 59.7 s, total: 1min 30s
Wall time: 2min 13s

EC2 instance:
It ran out of memory and the Kernel of Jupyter died.
I ran $htop to see the server condition.
both 2 cores of the CPU were using their upmost capacity and
the 8 GIB of RAM was compeletly full.

I tried to install different EC2 instances, including: linux-t3-xlarge, linux-t3-2xlarge, and linux-t3-3xlarge.
Due to limitation, we only have access to such server with 8 GIB of Ram. However, this task requires a server with 16 GIB of RAM and at least 4 cores of CPU.

![](./t3-large.png)

# ADM Homework 2 - Algorithmic Question (AQ)

In [ ]:
def calculate_pops(left_shelf, right_shelf, book_id):
    # Check if the book is on the left shelf.
    if book_id in left_shelf:
        # If the book is on the left shelf, find its index.
        left_pops = left_shelf.index(book_id)  # O(n), where n is the number of books on the left shelf
    else:
        # If the book is not on the left shelf, set left_pops to the number of books on the left shelf.
        left_pops = len(left_shelf)  # O(1)

    # Check if the book is on the right shelf.
    if book_id in right_shelf:
        # If the book is on the right shelf, find its index.
        right_pops = len(right_shelf) - right_shelf.index(book_id) - 1  # O(n), where n is the number of books on the right shelf
    else:
        # If the book is not on the right shelf, set right_pops to the number of books on the right shelf.
        right_pops = len(right_shelf)  # O(1)

    # Calculate the minimum pops between left_pops and right_pops.
    minimum_pops = min(left_pops, right_pops)  # O(1)

    # Return the minimum pops value.
    return minimum_pops
# Overall time complexity is O(n) in the worst case

def print_instruction_result():
    # Initialize the left and right shelves.
    left_shelf = []  # O(1)
    right_shelf = []  # O(1)
    
    # Read the number of instructions.
    n = int(input("How many instructions do you want to apply? "))  # O(1)

    # Initialize a list to store the results of type 3 instructions.
    results = []
    
    # Process each instruction.
    for i in range(n):  # O(n)
        # Read the instruction and split it into action and book_id.
        instruction = input(f"instruction # {i+1}: ").split()
        action, book_id = instruction[0], int(instruction[1])

        if action == 'L':
            # If it's 'L', insert the book at the beginning of the left shelf.
            left_shelf.insert(0, book_id)  # O(n) - Total number of instructions is 'n'.

        elif action == 'R':
            # If it's 'R', append the book to the right shelf.
            right_shelf.append(book_id)  # O(1)

        else:
            # we calculate the minimum pops, which involves searching for the book's position in both the left and right shelf lists. 
            # The worst-case time complexity for searching in a list is O(n), and in the worst case, we perform two searches. 
            # Therefore, the time complexity for calculating minimum pops is O(n).
            
            # Type 3 instruction, calculate and append the result to the results list.
            # Calculate the minimum pops by calling the 'calculate_pops' function.
            result = calculate_pops(left_shelf, right_shelf, book_id)  # O(n) - Searching in lists with worst-case time complexity.

            # Append the result to the results list.
            results.append(result)
    
    # Print the results for type 3 instructions.
    print(f"The results of {n} instructions are: ")

    # Iterate through the results list and print each result.
    for result in results:  # O(n)
        print(result)


In [ ]:
print_instruction_result()
# Input 1

# 8
# L 75
# R 20
# R 30
# L 11
# ? 75
# L 12
# L 15
# ? 20

# Output 1

# 1
# 1

How many instructions do you want to apply?  8
instruction # 1:  L 75
instruction # 2:  R 20
instruction # 3:  R 30
instruction # 4:  L 11
instruction # 5:  ? 75
instruction # 6:  L 12
instruction # 7:  L 15
instruction # 8:  ? 20


The results of 8 instructions are: 
1
1


In [ ]:
print_instruction_result()

# Input 2

# 17
# R 1
# L 2
# L 3
# L 4
# ? 3
# R 5
# R 6
# L 7
# L 8
# ? 4
# L 9
# R 10
# R 11
# L 12
# L 13
# ? 11
# ? 3

# Output 2:

# 1
# 2
# 0
# 5

How many instructions do you want to apply?  17
instruction # 1:  R 1
instruction # 2:  L 2
instruction # 3:  L 3
instruction # 4:  L 4
instruction # 5:  ? 3
instruction # 6:  R 5
instruction # 7:  R 6
instruction # 8:  L 7
instruction # 9:  L 8
instruction # 10:  ? 4
instruction # 11:  L 9
instruction # 12:  R 10
instruction # 13:  R 11
instruction # 14:  L 12
instruction # 15:  L 13
instruction # 16:  ? 11
instruction # 17:  ? 3


The results of 17 instructions are: 
1
2
0
5


## Calculate the time complexity (the Big O notation):

The overall time complexity of the print_instruction_result function is dominated by the loop iterating 'n' times, and the time complexity is $O(n^2)$ in the worst case. This is due to the $O(n)$ time complexity for inserting into the left shelf for each 'L' instruction, and the $O(n)$ time complexity for calculating minimum pops in '3' instructions. 

Overall time complexity (the Big $O$ notation) is the maximum time complexity of the whole code. which is : $O(n^2)$

## Is the proposed algorithm optimal?

The algorithm proposed in the minimum_pops function is not the most optimal one.
it performs a linear search in both left_shelf and right_shelf to find the position of the book with 'book_id'.
A more efficient approach would be using a dictionary or a set, to keep track of the positions of books in each shelf.
The following code is a revised minimum_pops method:

In [ ]:
def minimum_pops(left_shelf, right_shelf, book_id):
    # Create dictionaries to store book positions in left_shelf and right_shelf.
    left_positions = {book: index for index, book in enumerate(left_shelf)}
    right_positions = {book: index for index, book in enumerate(right_shelf)}
    
    # Find the position of the book with book_id in left_shelf, default to len(left_shelf) if not found.
    left_pops = left_positions.get(book_id, len(left_shelf))
    
    # Find the position of the book with book_id in right_shelf, default to len(right_shelf) if not found.
    right_pops = right_positions.get(book_id, len(right_shelf))
    
    # Calculate the minimum pops between left_pops and right_pops.
    minimum_pops = min(left_pops, right_pops)
    
    return minimum_pops


This revised code results in an overall time complexity of $O(1)$ for the minimum_pops function.

So in this case, the overall time complexity (the Big $O$ notation) of the whole code would be : $O(n)$